<a href="https://colab.research.google.com/github/WadeShadow/nlp2021/blob/main/Lab4/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

folder = '/content/drive/MyDrive/Colab Notebooks/NLP_labs/'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [5]:
RL_file = open(folder+'sentences.txt')
RL_sentences = RL_file.read().split('. ')[:-1]

DARINA_file = open(folder+'100.txt', encoding='windows-1251')
DARINA_sentences = DARINA_file.read().split('. ')[:-1]


max_tokens = 10000

sentences = RL_sentences+DARINA_sentences
tokens_count = 0
for sent in sentences:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(sentences))
avg_tokens


RL_sentences

['Training a virtual agent to outperform human players can teach us how to optimize different processes in a variety of different and exciting subfields',
 'This is what Google DeepMind did with its popular AlphaGo, which beat the strongest Go player in history and scored a goal that was considered impossible at the time.In this article, we will develop an AI agent that is able to learn how to play the popular game Snake from scratch',
 'To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow and PyTorch (both versions are available, you can choose the one you prefer)',
 'This approach consists in the interaction between two components: an environment (the game itself) and an agent (Snake)',
 'The agent collects information about its current state (we will see later what this means) and performs an action accordingly',
 'The environment rewards or punishes the agent based on the performed action',
 'Over time, the agent learns what actions m

In [6]:
sent_df = []

for sent in RL_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Reinforcement Learning'})

for sent in DARINA_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Neural network module for recognizing brain diseases'})

sent_df = pd.DataFrame(data=sent_df, columns=['Sentence', 'Target'])

sent_df['Label'] = sent_df['Target']
sent_df.replace({'Label':{'Reinforcement Learning':0, 'Neural network module for recognizing brain diseases':1}}, inplace=True)
sent_df

,Sentence,Target,Label
0,Training a virtual agent to outperform human p...,Reinforcement Learning,0
1,This is what Google DeepMind did with its popu...,Reinforcement Learning,0
2,"To do it, we implement a Deep Reinforcement Le...",Reinforcement Learning,0
3,This approach consists in the interaction betw...,Reinforcement Learning,0
4,The agent collects information about its curre...,Reinforcement Learning,0
...,...,...,...
211,This includes balancing the prospective benefi...,Neural network module for recognizing brain di...,1
212,"Users report their symptoms into the app, whic...",Neural network module for recognizing brain di...,1
213,Most common users could be people with serious...,Neural network module for recognizing brain di...,1
214,Body scans can spot cancer and vascular diseas...,Neural network module for recognizing brain di...,1


#### Tokenization&Embedding

In [8]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(sent_df['Sentence'])

text_vectorizer(sent_df['Sentence'])

<tf.Tensor: shape=(216, 23), dtype=int64, numpy=
array([[ 108,    6,  340, ...,  120,    4, 1214],
       [  27,    8,   47, ...,    6,   76,    9],
       [   3,  152,   26, ...,  618,   22,  216],
       ...,
       [  86,  314,  342, ...,    7,   64,  126],
       [1387,  794,   20, ..., 1159,    0,    0],
       [ 244,    2,  119, ...,  446,    8,    2]])>

In [9]:
embedding = layers.Embedding(input_dim=max_tokens, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=avg_tokens) 

embedding(text_vectorizer(sent_df['Sentence']))

<tf.Tensor: shape=(216, 23, 128), dtype=float32, numpy=
array([[[ 0.03660778, -0.03774943, -0.01531668, ...,  0.00106687,
         -0.01109964, -0.0175814 ],
        [ 0.00985781, -0.01773777, -0.00032331, ..., -0.02691876,
          0.00881269,  0.04212124],
        [ 0.00531664,  0.01032578,  0.02374015, ...,  0.02318429,
          0.04353384,  0.02461844],
        ...,
        [-0.03339157,  0.02094499, -0.02508782, ...,  0.04788938,
          0.01205983, -0.04124683],
        [-0.04488761, -0.00199626,  0.03677908, ...,  0.01848635,
          0.00675499, -0.03606325],
        [ 0.00984437, -0.03464404, -0.01386414, ...,  0.0416661 ,
          0.04685506, -0.04509468]],

       [[ 0.00233417, -0.01223688, -0.01157024, ...,  0.01490691,
          0.04043353, -0.01441158],
        [ 0.0464305 ,  0.03758707, -0.0402678 , ..., -0.00282789,
         -0.00543243,  0.01391425],
        [-0.01161529, -0.00038628, -0.04502564, ..., -0.00529779,
          0.01073959, -0.01220355],
        ...

#### Train/Test Split

In [32]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(sent_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Target,Label
14,What does this mean? It means that we need to ...,Reinforcement Learning,0
176,Integration issues have been a greater barrie...,Neural network module for recognizing brain di...,1
166,Although there is wisdom of excessive prospect...,Neural network module for recognizing brain di...,1
72,deepsense.ai took part in the “Learning to run...,Reinforcement Learning,0
169,This has produced remarkable enthusiasm in sev...,Neural network module for recognizing brain di...,1
...,...,...,...
54,An interesting upgrade might be obtained passi...,Reinforcement Learning,0
129,"The entire diagnostic process, in general, is ...",Neural network module for recognizing brain di...,1
97,But the same move ended up turning the tide in...,Reinforcement Learning,0
196,"Either way, increasing transparency in the sho...",Neural network module for recognizing brain di...,1


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: Naive Bayes Model, Dense Neural Network

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_NB = Pipeline([
                    ("tfidf", TfidfVectorizer()), 
                    ("clf", MultinomialNB()) 
])

# Fit the pipeline to the training data
model_NB.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_NB.score(test_split['Sentence'], test_split['Label'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 93.18%


### LSTM

In [39]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_lstm = tf.keras.Model(inputs, outputs, name="model_LSTM")


In [40]:
model_lstm.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [41]:
model_lstm.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                        

In [42]:
model_lstm_history = model_lstm.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=4,
                              validation_data=(test_split['Sentence'], test_split['Label']))


Epoch 1/4
6/6 [==============================] - 3s 207ms/step - loss: 0.6835 - accuracy: 0.6919 - val_loss: 0.6476 - val_accuracy: 0.9318
Epoch 2/4
6/6 [==============================] - 0s 19ms/step - loss: 0.6123 - accuracy: 0.9477 - val_loss: 0.5343 - val_accuracy: 0.9545
Epoch 3/4
6/6 [==============================] - 0s 20ms/step - loss: 0.4298 - accuracy: 0.9884 - val_loss: 0.2717 - val_accuracy: 0.9773
Epoch 4/4
6/6 [==============================] - 0s 20ms/step - loss: 0.1429 - accuracy: 0.9942 - val_loss: 0.1224 - val_accuracy: 0.9773


In [43]:
model_lstm_pred_probs = model_lstm.predict(test_split['Sentence'])
model_lstm_preds = tf.squeeze(tf.round(model_lstm_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_lstm_preds)


{'accuracy': 97.72727272727273,
 'f1': 0.9773081096381563,
 'precision': 0.9783549783549784,
 'recall': 0.9772727272727273}

### GRU

In [52]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x) 
x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_gru = tf.keras.Model(inputs, outputs, name="model_GRU")


In [53]:
model_gru.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [54]:
model_gru_history = model_gru.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=5,
                              validation_data=(test_split['Sentence'], test_split['Label']))


Epoch 1/5
6/6 [==============================] - 3s 141ms/step - loss: 0.6733 - accuracy: 0.7907 - val_loss: 0.6472 - val_accuracy: 0.7727
Epoch 2/5
6/6 [==============================] - 0s 21ms/step - loss: 0.6147 - accuracy: 0.8256 - val_loss: 0.5809 - val_accuracy: 0.8409
Epoch 3/5
6/6 [==============================] - 0s 21ms/step - loss: 0.5168 - accuracy: 0.8430 - val_loss: 0.4631 - val_accuracy: 0.9091
Epoch 4/5
6/6 [==============================] - 0s 19ms/step - loss: 0.3446 - accuracy: 0.9070 - val_loss: 0.2651 - val_accuracy: 0.9318
Epoch 5/5
6/6 [==============================] - 0s 22ms/step - loss: 0.1167 - accuracy: 0.9826 - val_loss: 0.0825 - val_accuracy: 0.9773


In [55]:
model_gru_pred_probs = model_gru.predict(test_split['Sentence'])
model_gru_preds = tf.squeeze(tf.round(model_gru_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_gru_preds)

{'accuracy': 97.72727272727273,
 'f1': 0.9773081096381563,
 'precision': 0.9783549783549784,
 'recall': 0.9772727272727273}

### Bidirectional LSTM

In [66]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # stacking RNN layers requires return_sequences=True
x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_lstm = tf.keras.Model(inputs, outputs, name="model_Bi_lstm")


In [67]:
model_bi_lstm.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [68]:
model_bi_lstm.summary()


Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_13 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [69]:
model_bi_lstm_history = model_bi_lstm.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=4,
                              validation_data=(test_split['Sentence'], test_split['Label']))


Epoch 1/4
6/6 [==============================] - 5s 235ms/step - loss: 0.6427 - accuracy: 0.8314 - val_loss: 0.5572 - val_accuracy: 0.9318
Epoch 2/4
6/6 [==============================] - 0s 26ms/step - loss: 0.4234 - accuracy: 1.0000 - val_loss: 0.3505 - val_accuracy: 0.9545
Epoch 3/4
6/6 [==============================] - 0s 26ms/step - loss: 0.1672 - accuracy: 1.0000 - val_loss: 0.1361 - val_accuracy: 0.9773
Epoch 4/4
6/6 [==============================] - 0s 27ms/step - loss: 0.0257 - accuracy: 1.0000 - val_loss: 0.0854 - val_accuracy: 0.9773


### Bidirectional GRU

In [71]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.GRU(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_gru = tf.keras.Model(inputs, outputs, name="model_Bi_gru")

In [72]:
model_bi_gru.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [73]:
model_bi_gru.summary()

Model: "model_Bi_gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,354,625
Trainable params: 1,354,625
No

In [74]:
model_bi_gru_history = model_bi_gru.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=10,
                              validation_data=(test_split['Sentence'], test_split['Label']))

Epoch 1/10
6/6 [==============================] - 6s 299ms/step - loss: 0.6228 - accuracy: 0.8081 - val_loss: 0.5573 - val_accuracy: 0.8409
Epoch 2/10
6/6 [==============================] - 0s 25ms/step - loss: 0.4598 - accuracy: 0.9593 - val_loss: 0.4301 - val_accuracy: 0.9091
Epoch 3/10
6/6 [==============================] - 0s 24ms/step - loss: 0.2831 - accuracy: 0.9709 - val_loss: 0.2852 - val_accuracy: 0.9318
Epoch 4/10
6/6 [==============================] - 0s 26ms/step - loss: 0.1106 - accuracy: 0.9942 - val_loss: 0.1535 - val_accuracy: 0.9318
Epoch 5/10
6/6 [==============================] - 0s 27ms/step - loss: 0.0194 - accuracy: 1.0000 - val_loss: 0.0944 - val_accuracy: 0.9545
Epoch 6/10
6/6 [==============================] - 0s 28ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0915 - val_accuracy: 0.9773
Epoch 7/10
6/6 [==============================] - 0s 27ms/step - loss: 4.7910e-04 - accuracy: 1.0000 - val_loss: 0.1087 - val_accuracy: 0.9773
Epoch 8/10
6/6 [======

## Здійснити порівняння ефективності моделей

### Naive Bayes

In [58]:
calculate_results(test_split['Label'], model_NB.predict(test_split['Sentence']))

{'accuracy': 93.18181818181817,
 'f1': 0.9316397887826459,
 'precision': 0.9324401913875597,
 'recall': 0.9318181818181818}

### LSTM

In [59]:
model_lstm_pred_probs = model_lstm.predict(test_split['Sentence'])
model_lstm_preds = tf.squeeze(tf.round(model_lstm_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_lstm_preds)

{'accuracy': 95.45454545454545,
 'f1': 0.9546395633352156,
 'precision': 0.9586776859504131,
 'recall': 0.9545454545454546}

### GRU

In [60]:
model_gru_pred_probs = model_gru.predict(test_split['Sentence'])
model_gru_preds = tf.squeeze(tf.round(model_gru_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_gru_preds)

{'accuracy': 97.72727272727273,
 'f1': 0.9773081096381563,
 'precision': 0.9783549783549784,
 'recall': 0.9772727272727273}

### BI_LSTM


In [78]:
model_bi_lstm_pred_probs = model_bi_lstm.predict(test_split['Sentence'])
model_bi_lstm_preds = tf.squeeze(tf.round(model_bi_lstm_pred_probs))
calculate_results(test_split['Label'], model_bi_lstm_preds)

{'accuracy': 95.45454545454545,
 'f1': 0.9546395633352156,
 'precision': 0.9586776859504131,
 'recall': 0.9545454545454546}

### BI_GRU

In [80]:
model_bi_gru_pred_probs = model_bi_gru.predict(test_split['Sentence'])
model_bi_gru_preds = tf.squeeze(tf.round(model_bi_gru_pred_probs))
calculate_results(test_split['Label'], model_bi_gru_preds)

{'accuracy': 97.72727272727273,
 'f1': 0.9773081096381563,
 'precision': 0.9783549783549784,
 'recall': 0.9772727272727273}